# common

In [ ]:
//// test

open testing

## common

### retry_fn'

In [ ]:
inl retry_fn' retries fn =
    let rec loop retry =
        inl is_error, result =
            match fn () with
            | Ok x => false, x
            | Error x => true, x
        if not is_error || retry >= retries
        then result
        else
            trace Debug
                fun () => "common.retry_fn\' / loop"
                fun () => { is_error retry = $'$"{!retry}/{!retries}"' : string; result }
            loop (retry + 1)
    loop 1

## fsharp

### upcast

In [ ]:
inl upcast forall t u. (x : t) : u =
    $'!x :> `u '

### downcast

In [ ]:
inl downcast forall t u. (x : t) : u =
    $'!x :?> `u '

### random

In [ ]:
nominal random = $'System.Random'

inl random () : random =
    $'`random ' ()

### random_next

In [ ]:
inl random_next (min : i32) (max : i32) (random : random) : i32 =
    $'!random.Next (!min, !max)'

### disposable

In [ ]:
nominal disposable t = $"backend_switch `({ Fsharp : $'System.IDisposable'; Python : $'object' })"

### dispose

In [ ]:
inl dispose (disposable : disposable _) : () =
    backend_switch {
        Fsharp = fun () => disposable |> $'_.Dispose()' : ()
        Python = fun () => $'!disposable.__exit__(None, None, None)' : ()
    }

### yield

In [ ]:
inl yield forall t. (x : t) : () =
    $'yield !x '

### return

In [ ]:
inl return forall t. (x : t) : () =
    $'return !x '

### return'

In [ ]:
inl return' forall t. (x : t) : t =
    $'return !x '

### retry_fn

In [ ]:
inl retry_fn forall t. retries (fn : () -> t) : option t =
    let rec loop retry =
        try
            fun () =>
                if retry < retries
                then fn () |> Some
                else None
            fun ex =>
                trace Warning
                    fun () => "common.retry_fn"
                    fun () => { retry ex }
                None
        |> function
            | Some x => x
            | None => loop (retry + 1)
    loop 0

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

inl retry_fn_test = mut 0i32
fun () =>
    retry_fn_test <- *retry_fn_test + 1
    *retry_fn_test
|> retry_fn 3i32
|> _assert_eq' (Some 1i32)

.py output (Cuda):
__assert_eq' / actual: US0_0(v0=1) / expected: US0_0(v0=1)

.rs output:
__assert_eq' / actual: US0_0(1) / expected: US0_0(1)

.ts output:
__assert_eq' / actual: US0_0 1 / expected: US0_0 1

.py output:
__assert_eq' / actual: US0_0 1 / expected: US0_0 1



.fsx output:
__assert_eq' / actual: US0_0 1 / expected: US0_0 1


In [ ]:
//// test
///! fsharp
////! cuda // v3 = $"retry: {v0} / ex: %A{v1} / {v2 ()}"
///! rust
///! typescript
///! python

inl retry_fn_test = mut 0i32
fun () =>
    if *retry_fn_test >= 2
    then *retry_fn_test
    else
        retry_fn_test <- *retry_fn_test + 1
        failwith "test"
|> retry_fn 3i32
|> _assert_eq' (Some 2i32)


.rs output:
00:00:00 w #1 common.retry_fn / { retry = 0; ex = Exception {
    message: "test",
} }
00:00:00 w #2 common.retry_fn / { retry = 1; ex = Exception {
    message: "test",
} }
__assert_eq' / actual: US0_0(2) / expected: US0_0(2)


.ts output:
00:00:00 w #1 common.retry_fn / { retry = 0; ex = Error: test }
00:00:00 w #2 common.retry_fn / { retry = 1; ex = Error: test }
__assert_eq' / actual: US0_0 2 / expected: US0_0 2


.py output:
00:00:00 w #1 common.retry_fn / { retry = 0; ex = test }
00:00:00 w #2 common.retry_fn / { retry = 1; ex = test }
__assert_eq' / actual: US0_0 2 / expected: US0_0 2




.fsx output:
00:00:00 w #1 common.retry_fn / { retry = 0; ex = System.Exception: test
   at FSI_0017.closure1(Mut0 v0, Int32 v1, Unit unitVar2)
   at FSI_0017.method1(Mut0 v0, Int32 v1) }
00:00:00 w #2 common.retry_fn / { retry = 1; ex = System.Exception: test
   at FSI_0017.closure1(Mut0 v0, Int32 v1, Unit unitVar2)
   at FSI_0017.method1(Mut0 v0, Int32 v1) }
__assert_eq' / actual: US0_0 2 / expected: US0_0 2


## common

### random'

In [ ]:
inl random' forall t. (min : t) (max : t) : t =
    run_target function
        | Rust (Contract) => fun () =>
            failwith "common.random' / target=Rust(Contract)"
        | Rust _ => fun () =>
            open rust.rust_operators
            !\\((min, max), $'"rand::Rng::gen_range(&mut rand::thread_rng(), $0..$1)"')
        | _ => fun () =>
            random () |> random_next (i32 min) (i32 max) |> convert

### new_disposable

In [ ]:
inl new_disposable (fn : () -> ()) : disposable _ =
    run_target function
        | Rust _ => fun () =>
            global "type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()"
            inl fn = join fn
            $'new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr !fn "$0()" )'
        | Fsharp _ | TypeScript _ | Python _ => fun () =>
            inl fn = join fn
            $'{ new System.IDisposable with member _.Dispose () = !fn () }'
        | Cuda _ => fun () =>
            $'class Disposable:'
            $'    def __init__(self, fn):'
            $'        self.fn = fn'
            $'    def __exit__(self, exc_type, exc_value, traceback):'
            $'        self.fn()'
            $'        return False'
            $'Disposable(!fn)'
        | _ => fun () => null ()

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

.py output (Cuda):
__assert_eq / actual: 1 / expected: 1

.rs output:
__assert_eq / actual: 1 / expected: 1

.ts output:
__assert_eq / actual: 1 / expected: 1

.py output:
__assert_eq / actual: 1 / expected: 1



.fsx output:
__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_task
|> async.await_task
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

__assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 0

__assert_eq / actual: 0 / expected: 0


## main

In [ ]:
inl main () =
    init_trace_state None
    inl new_disposable x : _ () = new_disposable x
    $'let new_disposable x = !new_disposable x' : ()
    inl retry_fn (r : i32) (x : () -> _) : optionm'.option' () = retry_fn r x |> optionm'.box
    $'let retry_fn x = !retry_fn x' : ()
    inl memoize (fn : () -> ()) : () -> () = memoize fn
    $'let memoize x = !memoize x' : ()